# Import required packages

In [13]:
# Optimization Packages
using JuMP
using SCS
using GLPK
using Gurobi

In [14]:
# Linear Algebra packages
using LinearAlgebra
using SparseArrays

# Part 3
Semi Definite Program
The problem at our hands is the following, we have an asymtotically stable LTI system. 
$$
\dot{x} = Ax + Dw \\
y = Cx
$$
where, $$
A = 
\begin{bmatrix}
0 & 1 & 0 \\
0 & 0 & 1 \\
-2 & -5.6 & -1 \\
\end{bmatrix} \\
D = 
\begin{bmatrix}
0  \\
0  \\
1 \\
\end{bmatrix} \\
C = 
\begin{bmatrix}
1 & 2 & 0.5
\end{bmatrix}.
$$
SDP problem that we are solving is the following
$$
\text{minimize} \:\:\: 0 \\
\text{subjected to} \:\:\: P \succ 0 \\
AP + PA^{T}+DD^{\ast}
$$
after solving this we would get value of $P$, using that we want to compute $\gamma = Trace(CPC^{\ast})$.

## Part 3.1 Creating a Model
Models are created with the `Model()` function. The way we create model and use is different from how we use GLPK package. 
- For discussion on SCS package refer the following [link](https://github.com/JuliaOpt/SCS.jl).

In [15]:
model = Model(solver=SCSSolver())
# model = Model(solver = GurobiSolver(Gurobi.Env(), OutputFlag = 0))

Academic license - for non-commercial use only


Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is Gurobi

## Part 3.2 Declaring Variables
We will create requried variables based on the above problem.
- For how to create JuMP variables [link](http://www.juliaopt.org/JuMP.jl/v0.19.0/variables/#Variables-1).
- For discussion on multidimensional arrays refer the following [link1](https://docs.julialang.org/en/v1/manual/arrays/), [link2](https://stanford.edu/class/ee103/julia_slides/julia_matrices_slides.pdf).
- For discussion on how to add SDP constraints refer the following [link0](http://www.juliaopt.org/JuMP.jl/v0.19.0/constraints/), [link1](http://www.juliaopt.org/JuMP.jl/v0.19.0/extensions/#Adding-add_constraint-methods-1), [link2](http://www.juliaopt.org/JuMP.jl/v0.12/refexpr.html).

Note that we can't add positive definite constraint in the optimization problem. Hence we will compute value $P$ such that it is greater than some positive constant $e$ i.e., we want $P \succeq e \implies P-e \succeq 0$. Since $e$ is a matrix with we have 

In [16]:
# Some constants that we need
A = [[0 1 0]; [0 0 1]; [-2 -5.6 -1]]
D = [[0]; [0]; [1]]
C = [1 2 0.5]
e = 1e-9*ones(3, 3) 
# e = I(3)
println("Size of the P matrix should be: ", size(A))

Size of the P matrix should be: (3, 3)


In [17]:
@variable(model, P[1:3, 1:3])

3×3 Array{Variable,2}:
 P[1,1]  P[1,2]  P[1,3]
 P[2,1]  P[2,2]  P[2,3]
 P[3,1]  P[3,2]  P[3,3]

## Part 3.3 Setting the Objective
Next we'll set our objective on the `model` we defined earlier. The objective sense, `Max` or `Min`, should be provided as the second argument. 

In [18]:
@objective(model, Min, 0)

0

## Part 3.4 Setting the Constraints
Adding constraints is a lot like setting the objective. Here we create a less-than-or-equal-to constraint using `<=`, but we can also create equality constraints using `==` and greater-than-or-equal-to constraints with `>=`.

In [19]:
@constraint(model, con_1, A*P + P*A' + D*D' .== 0)

3×3 Array{ConstraintRef{Model,JuMP.GenericRangeConstraint{JuMP.GenericAffExpr{Float64,Variable}}},2}:
 P[2,1] + P[1,2] == 0                           …  P[2,3] - 2 P[1,1] - 5.6 P[1,2] - P[1,3] == 0                   
 P[3,1] + P[2,2] == 0                              P[3,3] - 2 P[2,1] - 5.6 P[2,2] - P[2,3] == 0                   
 -2 P[1,1] - 5.6 P[2,1] - P[3,1] + P[3,2] == 0     -2 P[1,3] - 5.6 P[2,3] - 2 P[3,3] - 2 P[3,1] - 5.6 P[3,2] == -1

In [20]:
# Both are valid ways of creating PSD constraints
# @constraint(model, (P-e) in PSDCone())
# @SDconstraint(model, P >= e)
@constraint(model, P in PSDCone())

 P[1,1]  P[1,2]  P[1,3]
 P[2,1]  P[2,2]  P[2,3] is semidefinite
 P[3,1]  P[3,2]  P[3,3]

## Part 3.5 Optimizing the objective
Models are solved with the `JuMP.optimize!` function. After the call to `JuMP.optimize!` has finished, we need to query what happened. The solve could terminate for a number of reasons. First, the solver might have found the optimal solution or proved that the problem is infeasible. However, it might also have run into numerical difficulties or terminated due to a setting such as a time limit. We can ask the solver why it stopped using the `JuMP.termination_status` function.

In [21]:
solve(model)

ErrorException: Cone type SDP not supported

## Part 3.6 Results

In [10]:
output = getvalue(P)
println("Optimal value of P such that is positive definite is:", "\n", output)

Optimal value of P such that is positive definite is:
[0.0694439866695286 6.637551341706084e-8 -0.13888881555687946; 6.637551341630075e-8 0.13888866971827424 1.447193690694444e-7; -0.13888881555687946 1.447193690827456e-7 0.7777777219537352]


In [11]:
eigen(output)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
3-element Array{Float64,1}:
 0.0431844182924201 
 0.13888866971833755
 0.8040372903307806 
eigenvectors:
3×3 Array{Float64,2}:
 -0.982592     9.81918e-7  -0.185778  
  9.62399e-7   1.0          1.95248e-7
 -0.185778    -1.3057e-8    0.982592  

In [12]:
gamma = tr(C * output * C')
print("Gamma: ", gamma)

Gamma: 0.6805548354149716